In [85]:
import random
import pandas as pd
import time
import logging
import datetime
import re

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import *
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from fake_useragent import UserAgent

import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

'C:\\Users\\Mattheus\\anaconda3\\envs\\smm750\\lib\\site-packages\\chromedriver_autoinstaller\\95\\chromedriver.exe'

In [104]:
def web_driver(headless=False):
    option = webdriver.ChromeOptions()
    if(headless == True):
        option.add_argument('--headless')
    option.add_argument('start-maximized')
    option.add_argument('disable-infobars')
    option.add_argument('--disable-extensions')
    driver = webdriver.Chrome(options=option)
    return driver

def scrape_doge_tweets():
    print('Starting Scrape...')
    tweet_urls = pd.read_csv('data/tweet_url.csv')
    to_parse = scrape_tweets(tweet_urls['tweet_url'])
    to_return = parse_tweets(to_parse)
    print('Scrape Success!')
    return to_return

def scrape_tweets(tweet_urls):
    to_return =[]
    
    for i in range(len(tweet_urls)):
        tweet_url = tweet_urls[i]
        print(f'Scraping URL {str(i+1)}/{str(len(tweet_urls))}: {tweet_url}')
        driver = web_driver()
        driver.get(tweet_url)
        driver.implicitly_wait(5)
        tweets = driver.find_elements_by_xpath('//*[@data-testid="tweet"]')

        for i in range(2):
            tweet = tweets[i].text
            if 'Likes' in tweet:
                to_return.append(tweet)
                break
            else:
                continue
        
        driver.quit()
        time.sleep(2)

    return to_return

def parse_tweets(tweets):
    tweet_contents=[];timestamps=[];retweets=[];quotes=[];likes=[]

    for i in range(len(tweets)):
        print(f'Parsing scraped data: {str(i+1)}/{str(len(tweets))}')
        tweet = tweets[i]
        # print(tweet)
        # print(len(tweet.split('\n')))
        
        time = tweet.split('\n')[-7].split('·')[0].strip()
        date = tweet.split('\n')[-7].split('·')[1].strip()
        timestamp = datetime.datetime.strptime(date + ' ' + time, '%b %d, %Y %I:%M %p')

        timestamps.append(timestamp)
        retweets.append(format_metric(tweet.split('\n')[-6]))
        quotes.append(format_metric(tweet.split('\n')[-4]))
        likes.append(format_metric(tweet.split('\n')[-2]))

        time_section = tweet.split('\n')[-7]
        tweet_contents.append(tweet.split('@elonmusk\n')[1].split('\n'+time_section)[0].replace('\n',' ').encode("ascii", "ignore").decode())
        # tweet_contents.append(tweet.split('\n')[2].strip().encode("ascii", "ignore").decode())

    to_return = pd.DataFrame(zip(timestamps,tweet_contents,retweets,quotes,likes), columns=['timestamp','tweet_content','retweet_count','quote_count','like_count'])
    # ,tweet_contents ,'tweet_content'
    return to_return

def format_metric(text):
    text = text.replace(',','')

    if 'K' in text:
        text = text.replace('K','')
        text = float(text) * 1000
    elif 'M' in text:
        text = text.replace('M','')
        text = float(text) * 1000000
    elif 'B' in text:
        text = text.replace('B','')
        text = float(text) * 1000000000
    else:
        text = float(text)
        
    return text

In [107]:
data = scrape_doge_tweets()
data

Starting Scrape...
Scraping URL 1/26: https://twitter.com/elonmusk/status/1454876031232380928
Scraping URL 2/26: https://twitter.com/elonmusk/status/1410529698497630212
Scraping URL 3/26: https://twitter.com/elonmusk/status/1410519466518233089
Scraping URL 4/26: https://twitter.com/elonmusk/status/1396916392629137409
Scraping URL 5/26: https://twitter.com/elonmusk/status/1396926365996261382
Scraping URL 6/26: https://twitter.com/elonmusk/status/1395328697436033032
Scraping URL 7/26: https://twitter.com/elonmusk/status/1392974251011895300
Scraping URL 8/26: https://twitter.com/elonmusk/status/1392030108274159619
Scraping URL 9/26: https://twitter.com/elonmusk/status/1391523807148527620
Scraping URL 10/26: https://twitter.com/elonmusk/status/1387290679794089986
Scraping URL 11/26: https://twitter.com/elonmusk/status/1382552587099062272
Scraping URL 12/26: https://twitter.com/elonmusk/status/1377567762919292938
Scraping URL 13/26: https://twitter.com/elonmusk/status/1370885153262817283
Sc

,timestamp,tweet_content,retweet_count,quote_count,like_count
0,2021-10-31 18:20:00,Tuition is in Dogecoin & u get a discount if u...,22600.0,3149.0,241200.0
1,2021-07-01 10:24:00,"Baby Doge, doo, doo, doo, doo, doo, Baby Doge,...",46600.0,10300.0,279000.0
2,2021-07-01 09:43:00,Release the Doge!,16300.0,1569.0,130500.0
3,2021-05-24 20:49:00,"If youd like to help develop Doge, please subm...",32500.0,3098.0,191300.0
4,2021-05-24 21:29:00,Someone suggested changing Dogecoin fees based...,12000.0,1049.0,130300.0
5,2021-05-20 11:41:00,How much is that Doge in the window?,53500.0,8099.0,307000.0
6,2021-05-13 23:45:00,Working with Doge devs to improve system trans...,84000.0,21000.0,541800.0
7,2021-05-11 09:13:00,Do you want Tesla to accept Doge? Yes 78.2% No...,96300.0,16200.0,395500.0
8,2021-05-09 23:41:00,SpaceX launching satellite Doge-1 to the moon ...,114200.0,15300.0,533600.0
9,2021-04-28 07:20:00,The Dogefather SNL May 8,76100.0,10700.0,453400.0


In [108]:
data.to_csv('data/doge_tweets.csv', index=False)

In [106]:
to_return

,timestamp,tweet_content,retweet_count,quote_count,like_count
0,2021-10-31 18:20:00,Tuition is in Dogecoin & u get a discount if u...,22600.0,3149.0,241200.0


In [76]:
tweet_urls = pd.read_csv('data/tweet_url.csv')
test = tweet_urls.head(2)['tweet_url']
test

0    https://twitter.com/elonmusk/status/1454876031...
1    https://twitter.com/elonmusk/status/1410529698...
Name: tweet_url, dtype: object

In [77]:
to_parse = scrape_tweets(test)

In [93]:
to_parse

['Elon Musk\n@elonmusk\nTuition is in Dogecoin & u get a discount if u have a dog\n6:20 PM · Oct 31, 2021·Twitter for iPhone\n22.6K\n Retweets\n3,148\n Quote Tweets\n241.2K\n Likes',
 'Elon Musk\n@elonmusk\nBaby Doge, doo, doo, doo, doo, doo,\nBaby Doge, doo, doo, doo, doo, doo,\nBaby Doge, doo, doo, doo, doo, doo,\nBaby Doge\n10:24 AM · Jul 1, 2021·Twitter for iPhone\n46.6K\n Retweets\n10.3K\n Quote Tweets\n279K\n Likes']

In [99]:
data = parse_tweets(to_parse)
# data.to_csv('data/doge_tweets.csv', index=False)
data

1/2
2/2


,timestamp,tweet_content,retweet_count,quote_count,like_count
0,2021-10-31 18:20:00,Tuition is in Dogecoin & u get a discount if u...,22600.0,3148.0,241200.0
1,2021-07-01 10:24:00,"Baby Doge, doo, doo, doo, doo, doo, Baby Doge,...",46600.0,10300.0,279000.0


## Import Packages

In [118]:
import requests as req
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import json
import logging
import datetime
from lxml import etree

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('wordnet')
import matplotlib.pyplot as plt
import spacy
import re

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import *
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from fake_useragent import UserAgent
import random
import time

import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mattheus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Define Functions

In [10]:
def scrape_reuters():
    headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}
    url ="https://www.reuters.com/technology/do-you-want-tesla-accept-doge-musk-asks-twitter-2021-05-11/"
    res = req.get(url, headers=headers)
    soup = BeautifulSoup(res.content, "html.parser")
    dom = etree.HTML(str(soup))
    content_list = dom.xpath('//*[contains(@class,"paywall-article")]//*')

    to_join = []
    for i in range(len(content_list)):
        text = str(content_list[i].text).strip()
        # print(text)
        if text != "None" and text != "read more":
            to_join.append(text)

    full_content = ' '.join(to_join)
    
    return full_content

In [119]:
def try_find_element_by_xpath(xpath_code):
    try:
        text = driver.find_element_by_xpath(xpath_code)
        if(text == ''):
            text = 'Unavailable'
        return text
    except NoSuchElementException:
        logger.info('NoSuchElementException')
        text = '-'
        return text
        
def try_webdriver_text(webdriver_object):
    try:
        text = webdriver_object.text
        if(text == ''):
            text = 'Unavailable'
        return text
    except AttributeError:
        logger.info('AttributeError')
        text = '-'
        return text
    
def try_get_attribute(element, string):
    try:
        text = element.get_attribute(string)
        if(text == ''):
            text = 'Unavailable'
        return text
    except AttributeError:
        logger.info('AttributeError')
        text = '-'
        return text
        
def web_driver(headless=False):
    option = webdriver.ChromeOptions()
    if(headless == True):
        option.add_argument('--headless')
    # ua = UserAgent()
    # userAgent = ua.random
    # logger.info(userAgent)
    # option.add_argument(f'user-agent={userAgent}')
    # option.add_argument(' — incognito')
    option.add_argument('start-maximized')
    option.add_argument('disable-infobars')
    option.add_argument('--disable-extensions')
    driver = webdriver.Chrome(options=option)
    return driver

In [120]:
tweet_urls = pd.read_csv('data/tweet_url.csv')
tweet_urls

,tweet_url
0,https://twitter.com/elonmusk/status/1340590280...
1,https://twitter.com/elonmusk/status/1357241340...
2,https://twitter.com/elonmusk/status/1357244295...


In [64]:
toAppend=[]

for tweet_url in tweet_urls['tweet_url']:
    driver = web_driver()
    driver.get(tweet_url)
    driver.implicitly_wait(5)
    tweet = driver.find_element_by_xpath('//*[@data-testid="tweet"]').text
    toAppend.append(tweet)
    driver.quit()
    time.sleep(2)

toAppend

['Elon Musk\n@elonmusk\nOne word: Doge\n9:30 AM · Dec 20, 2020·Twitter for iPhone\n25.1K\n Retweets\n6,862\n Quote Tweets\n216.6K\n Likes',
 'Elon Musk\n@elonmusk\nDogecoin is the people’s crypto\n8:15 AM · Feb 4, 2021·Twitter Web App\n100K\n Retweets\n13.6K\n Quote Tweets\n541.5K\n Likes',
 'Elon Musk\n@elonmusk\nNo highs, no lows, only Doge\n8:27 AM · Feb 4, 2021·Twitter Web App\n109.4K\n Retweets\n11.3K\n Quote Tweets\n755.7K\n Likes']

In [121]:
def web_driver(headless=False):
    option = webdriver.ChromeOptions()
    if(headless == True):
        option.add_argument('--headless')
    # ua = UserAgent()
    # userAgent = ua.random
    # logger.info(userAgent)
    # option.add_argument(f'user-agent={userAgent}')
    # option.add_argument(' — incognito')
    option.add_argument('start-maximized')
    option.add_argument('disable-infobars')
    option.add_argument('--disable-extensions')
    driver = webdriver.Chrome(options=option)
    return driver

def scrape_doge_tweets():
    tweet_urls = pd.read_csv('data/tweet_url.csv')
    to_parse = scrape_tweets(tweet_urls['tweet_url'])
    to_return = parse_tweets(to_parse)

    return to_return

def scrape_tweets(tweet_urls):
    to_return =[]

    for tweet_url in tweet_urls:
        driver = web_driver()
        driver.get(tweet_url)
        driver.implicitly_wait(5)
        tweet = driver.find_element_by_xpath('//*[@data-testid="tweet"]').text
        to_return.append(tweet)
        driver.quit()
        time.sleep(2)

    return to_return

def parse_tweets(tweets):
    tweet_contents=[];timestamps=[];retweets=[];quotes=[];likes=[]
    
    for tweet in tweets:
        tweet_contents.append(tweet.split('\n')[2].strip())

        time = tweet.split('\n')[3].split('·')[0].strip()
        date = tweet.split('\n')[3].split('·')[1].strip()
        timestamp = datetime.datetime.strptime(date + ' ' + time, '%b %d, %Y %I:%M %p')

        timestamps.append(timestamp)
        retweets.append(format_metric(tweet.split('\n')[4]))
        quotes.append(format_metric(tweet.split('\n')[6]))
        likes.append(format_metric(tweet.split('\n')[8]))

    to_return = pd.DataFrame(zip(timestamps,tweet_contents,retweets,quotes,likes), columns=['timestamp','tweet_content','retweet_count','quote_count','like_count'])
    
    return to_return

def format_metric(text):
    text = text.replace(',','')

    if 'K' in text:
        text = text.replace('K','')
        text = float(text) * 1000
    elif 'M' in text:
        text = text.replace('M','')
        text = float(text) * 1000000
    elif 'B' in text:
        text = text.replace('B','')
        text = float(text) * 1000000000
    else:
        text = float(text)
        
    return text

In [122]:
data = scrape_doge_tweets()

In [123]:
data

,timestamp,tweet_content,retweet_count,quote_count,like_count
0,2020-12-20 09:30:00,One word: Doge,25100.0,6862.0,216600.0
1,2021-02-04 08:15:00,Dogecoin is the people’s crypto,100000.0,13600.0,541500.0
2,2021-02-04 08:27:00,"No highs, no lows, only Doge",109400.0,11300.0,755600.0


In [114]:
tweet_contents=[];timestamps=[];retweets=[];quotes=[];likes=[]
for tweet in toAppend:
    tweet_contents.append(tweet.split('\n')[2].strip())

    time = tweet.split('\n')[3].split('·')[0].strip()
    date = tweet.split('\n')[3].split('·')[1].strip()
    timestamp = datetime.datetime.strptime(date + ' ' + time, '%b %d, %Y %I:%M %p')

    timestamps.append(timestamp)
    retweets.append(format_metric(tweet.split('\n')[4]))
    quotes.append(format_metric(tweet.split('\n')[6]))
    likes.append(format_metric(tweet.split('\n')[8]))

toDf = pd.DataFrame(zip(timestamps,tweet_contents,retweets,quotes,likes), columns=['timestamp','tweet_content','retweet_count','quote_count','like_count'])
toDf

['Elon Musk', '@elonmusk', 'One word: Doge', '9:30 AM · Dec 20, 2020·Twitter for iPhone', '25.1K', ' Retweets', '6,862', ' Quote Tweets', '216.6K', ' Likes']
['Elon Musk', '@elonmusk', 'Dogecoin is the people’s crypto', '8:15 AM · Feb 4, 2021·Twitter Web App', '100K', ' Retweets', '13.6K', ' Quote Tweets', '541.5K', ' Likes']
['Elon Musk', '@elonmusk', 'No highs, no lows, only Doge', '8:27 AM · Feb 4, 2021·Twitter Web App', '109.4K', ' Retweets', '11.3K', ' Quote Tweets', '755.7K', ' Likes']


## Topic Modeling

In [3]:
content = scrape_reuters()

In [4]:
content

'May 11 (Reuters) - Billionaire Elon Musk and cryptocurrency aficionado on Tuesday asked The question, in the form of a Twitter poll, comes days after he called dogecoin a "hustle," sending the meme-inspired cryptocurrency\'s price reeling after a 700% rally in a month. A proponent of digital currencies, the Tesla Inc (TSLA.O) Musk is one of the world\'s richest people and owner of several futuristic companies, including SpaceX and Neuralink. He has used his candid Twitter feed to give his opinion on cryptocurrencies, including bitcoin, most times affecting their price. In March, Musk said U.S. customers could purchase Tesla vehicles with bitcoin, marking a significant step forward for the cryptocurrency\'s use in commerce. The electric-car maker had bought $1.5 billion worth of bitcoin earlier this year, propelling its prices to record highs. Based on current prices, one would need nearly 80,000 dogecoins or 0.7 bitcoin to buy the cheapest Tesla Model 3 car. 1/2 SpaceX owner and Tesla

In [26]:
nlp = spacy.load('en_core_web_sm') # we first load the English language model
parsed_content = nlp(content)
print (parsed_content)

May 11 (Reuters) - Billionaire Elon Musk and cryptocurrency aficionado on Tuesday asked The question, in the form of a Twitter poll, comes days after he called dogecoin a "hustle," sending the meme-inspired cryptocurrency's price reeling after a 700% rally in a month. A proponent of digital currencies, the Tesla Inc (TSLA.O) Musk is one of the world's richest people and owner of several futuristic companies, including SpaceX and Neuralink. He has used his candid Twitter feed to give his opinion on cryptocurrencies, including bitcoin, most times affecting their price. In March, Musk said U.S. customers could purchase Tesla vehicles with bitcoin, marking a significant step forward for the cryptocurrency's use in commerce. The electric-car maker had bought $1.5 billion worth of bitcoin earlier this year, propelling its prices to record highs. Based on current prices, one would need nearly 80,000 dogecoins or 0.7 bitcoin to buy the cheapest Tesla Model 3 car. 1/2 SpaceX owner and Tesla CEO

In [28]:
# take out individual tokens from parsed review
token_text = [token.text for token in parsed_content]
print(token_text)

['May', '11', '(', 'Reuters', ')', '-', 'Billionaire', 'Elon', 'Musk', 'and', 'cryptocurrency', 'aficionado', 'on', 'Tuesday', 'asked', 'The', 'question', ',', 'in', 'the', 'form', 'of', 'a', 'Twitter', 'poll', ',', 'comes', 'days', 'after', 'he', 'called', 'dogecoin', 'a', '"', 'hustle', ',', '"', 'sending', 'the', 'meme', '-', 'inspired', 'cryptocurrency', "'s", 'price', 'reeling', 'after', 'a', '700', '%', 'rally', 'in', 'a', 'month', '.', 'A', 'proponent', 'of', 'digital', 'currencies', ',', 'the', 'Tesla', 'Inc', '(', 'TSLA.O', ')', 'Musk', 'is', 'one', 'of', 'the', 'world', "'s", 'richest', 'people', 'and', 'owner', 'of', 'several', 'futuristic', 'companies', ',', 'including', 'SpaceX', 'and', 'Neuralink', '.', 'He', 'has', 'used', 'his', 'candid', 'Twitter', 'feed', 'to', 'give', 'his', 'opinion', 'on', 'cryptocurrencies', ',', 'including', 'bitcoin', ',', 'most', 'times', 'affecting', 'their', 'price', '.', 'In', 'March', ',', 'Musk', 'said', 'U.S.', 'customers', 'could', 'purc

In [29]:
# lemmatize words
token_lemmas = [token.lemma_ for token in parsed_content]
print(token_lemmas)

['May', '11', '(', 'Reuters', ')', '-', 'Billionaire', 'Elon', 'Musk', 'and', 'cryptocurrency', 'aficionado', 'on', 'Tuesday', 'ask', 'the', 'question', ',', 'in', 'the', 'form', 'of', 'a', 'Twitter', 'poll', ',', 'come', 'day', 'after', 'he', 'call', 'dogecoin', 'a', '"', 'hustle', ',', '"', 'send', 'the', 'meme', '-', 'inspire', 'cryptocurrency', "'s", 'price', 'reeling', 'after', 'a', '700', '%', 'rally', 'in', 'a', 'month', '.', 'a', 'proponent', 'of', 'digital', 'currency', ',', 'the', 'Tesla', 'Inc', '(', 'TSLA.O', ')', 'Musk', 'be', 'one', 'of', 'the', 'world', "'s", 'rich', 'people', 'and', 'owner', 'of', 'several', 'futuristic', 'company', ',', 'include', 'spacex', 'and', 'Neuralink', '.', 'he', 'have', 'use', 'his', 'candid', 'Twitter', 'feed', 'to', 'give', 'his', 'opinion', 'on', 'cryptocurrencie', ',', 'include', 'bitcoin', ',', 'most', 'time', 'affect', 'their', 'price', '.', 'in', 'March', ',', 'Musk', 'say', 'U.S.', 'customer', 'could', 'purchase', 'Tesla', 'vehicle', '

In [30]:
# check if token is a stop word
token_stop = [token.is_stop for token in parsed_content]
for t in range(len(token_lemmas)):
    if token_stop[t]:
        print("Token '" + token_lemmas[t] + "' is a stopword")
    else:
        print("Token '" + token_lemmas[t] + "' is not a stopword")

Token 'May' is a stopword
Token '11' is not a stopword
Token '(' is not a stopword
Token 'Reuters' is not a stopword
Token ')' is not a stopword
Token '-' is not a stopword
Token 'Billionaire' is not a stopword
Token 'Elon' is not a stopword
Token 'Musk' is not a stopword
Token 'and' is a stopword
Token 'cryptocurrency' is not a stopword
Token 'aficionado' is not a stopword
Token 'on' is a stopword
Token 'Tuesday' is not a stopword
Token 'ask' is not a stopword
Token 'the' is a stopword
Token 'question' is not a stopword
Token ',' is not a stopword
Token 'in' is a stopword
Token 'the' is a stopword
Token 'form' is not a stopword
Token 'of' is a stopword
Token 'a' is a stopword
Token 'Twitter' is not a stopword
Token 'poll' is not a stopword
Token ',' is not a stopword
Token 'come' is not a stopword
Token 'day' is not a stopword
Token 'after' is a stopword
Token 'he' is a stopword
Token 'call' is not a stopword
Token 'dogecoin' is not a stopword
Token 'a' is a stopword
Token '"' is not 

In [31]:
# assemble results for inspection
pd.DataFrame(zip(token_text, token_lemmas, token_stop), columns=['Original Text', 'Stemmed Text', 'stopwords']).head(10)

,Original Text,Stemmed Text,stopwords
0,May,May,True
1,11,11,False
2,(,(,False
3,Reuters,Reuters,False
4,),),False
5,-,-,False
6,Billionaire,Billionaire,False
7,Elon,Elon,False
8,Musk,Musk,False
9,and,and,True
